In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 配置

In [4]:
# 模型保存路径
model_path = "../model/"
# 数据路径
data_path = "../data/corpus/ass.out"
seq2seq_model_name = "seq2seq.h5"

# 模型测试

In [39]:
import seq2seq_model
from imp import reload
reload(seq2seq_model)
tf.contrib.eager.enable_eager_execution()

x = np.random.normal(size=(100, 100, 10))
y = np.random.normal(size=(100, 100, 10))
z = np.random.normal(size=(100, 100, 10))

input_dim = 10
encoder_units = [10,20,30]
decoder_units = [10,20,30]
hidden_units = [40,40]
batch_size = 10


model = seq2seq_model.build_seq2seq(
    input_dim=input_dim,
    encoder_units=encoder_units,
    decoder_units=decoder_units,
    batch_size=batch_size,
    hidden_units=hidden_units
)
# model.compile(optimizer=tf.train.AdamOptimizer(), loss='categorical_crossentropy')
# model.fit([x, y], z, batch_size = 10)


ValueError: tf.enable_eager_execution must be called at program startup.

In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder_Input (InputLayer)      (None, None, 10)     0                                            
__________________________________________________________________________________________________
Decoder_Input (InputLayer)      (10, None, 10)       0                                            
__________________________________________________________________________________________________
Encoder_LSTM_1 (CuDNNLSTM)      [(None, None, 10), ( 880         Encoder_Input[0][0]              
__________________________________________________________________________________________________
Decoder_LSTM_1 (CuDNNLSTM)      [(10, None, 10), (10 880         Decoder_Input[0][0]              
                                                                 Encoder_LSTM_1[0][1]             
          

In [8]:
model.compile(optimizer=tf.train.AdamOptimizer(), loss='categorical_crossentropy')
model.fit([x, y], z, batch_size = 10)

Epoch 1/1
10/10 [==============================] - 8s 751ms/step - loss: 0.0152


# 预处理

In [3]:
!head "../data/corpus/ass.out"

'head' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


## 分词

In [40]:
with open(data_path, encoding="utf-8") as f:
    for i,v in enumerate(f):
        print(v)
        if i == 20:
            break

:美战之星字幕组

月光水晶城

翻 译

明月

之心

校 对

心月

晓雪

时间轴制作

飞天

圣明

特 效

心月

晓雪

监督

新月

冰冰

小小兔变得有精神了

而我却还没能喘口气

她就又跑来缠着小卫了

心有不甘的我去狂吃蛋糕



In [41]:
samples = 1000
t = list()
with open(data_path, encoding="utf-8") as f:
    for i, v in enumerate(f):
        if i == samples:
            break
        t.append(v)  



In [42]:
import jieba
t1 = list()

for v in t:
    words = jieba.cut(v)
    t1.append(" ".join(words))

In [43]:
from tensorflow.keras.preprocessing import text as T
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=None)
tokenizer.fit_on_texts(t1)
t2 = tokenizer.texts_to_sequences(t1)
x = pad_sequences(t2,maxlen=100,padding="post")
x

array([[ 259,  260,  261, ...,    0,    0,    0],
       [ 113,  366,    0, ...,    0,    0,    0],
       [ 262,  195,    0, ...,    0,    0,    0],
       ...,
       [  73,    6,    0, ...,    0,    0,    0],
       [  73, 1329,    5, ...,    0,    0,    0],
       [  11,   11,   32, ...,    0,    0,    0]])

## 训练模型

In [44]:
len(tokenizer.word_counts)

1330

In [54]:
from imp import reload
import seq2seq_model

batch_size = 10
encoder_units = [128,128,128]
decoder_units = encoder_units
voc_size = len(tokenizer.word_counts)
hidden_units = [128,64]
input_dim = voc_size
seq_length = 100
epoch = 10

reload(seq2seq_model)
model = seq2seq_model.build_seq2seq(
                    batch_size=batch_size,
                    encoder_units=encoder_units,
                    decoder_units=decoder_units,
                    voc_size=voc_size,
                    input_dim=input_dim,
                    seq_length = seq_length,
                    hidden_units=hidden_units
)

In [46]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder_Input (InputLayer)      (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     399000      Encoder_Input[0][0]              
__________________________________________________________________________________________________
Decoder_Input (InputLayer)      (10, None, 1330)     0                                            
__________________________________________________________________________________________________
Encoder_LSTM_1 (CuDNNLSTM)      [(None, 100, 128), ( 220160      embedding_1[0][0]                
__________________________________________________________________________________________________
Decoder_LS

In [47]:
data, label = list(),list()
for i,seq in enumerate(x):
    if i % 2 == 0:
        data.append(seq)
    else:
        label.append(seq)

data = np.asarray(data)
label = np.asarray(label)

len(data),len(label)

(500, 500)

In [48]:
import copy

x2 = list()
for i, v in enumerate(label):
    x2_v = copy.copy(v)
    x2_v = x2_v.tolist()
    x2_v.insert(0,0)
    x2_v = x2_v[:seq_length]
    x2.append(x2_v)
    
x2 = np.asarray(x2)

In [49]:
from sklearn.preprocessing import OneHotEncoder

def one_hot_encode(data,voc_size):
    """one_hot编码数据"""
    data_length,seq_length = data.shape
    enc = OneHotEncoder(n_values=voc_size,sparse=False)
    t = enc.fit_transform(data)
    t.shape = (data_length,seq_length,voc_size)
    return t

In [50]:
flag = True
if flag:
    x2 = one_hot_encode(x2,voc_size)
    y = one_hot_encode(label,voc_size)

In [51]:
x1 = data
x1.shape,x2.shape,y.shape

((500, 100), (500, 100, 1330), (500, 100, 1330))

In [55]:

epoch = 10
model.compile(optimizer=tf.train.AdamOptimizer(), loss='categorical_crossentropy')
model.fit([x1,x2],y,batch_size=batch_size,epochs=epoch,validation_split=0.1)

Train on 450 samples, validate on 50 samples
Epoch 1/10
450/450 [==============================] - 7s 16ms/step - loss: 2.3624 - val_loss: 0.3561
Epoch 2/10
450/450 [==============================] - 5s 11ms/step - loss: 0.3647 - val_loss: 0.3437
Epoch 3/10
450/450 [==============================] - 5s 11ms/step - loss: 0.3541 - val_loss: 0.3508
Epoch 4/10
450/450 [==============================] - 5s 11ms/step - loss: 0.3476 - val_loss: 0.3599
Epoch 5/10
450/450 [==============================] - 5s 11ms/step - loss: 0.3451 - val_loss: 0.3625
Epoch 6/10
450/450 [==============================] - 5s 11ms/step - loss: 0.3434 - val_loss: 0.3759
Epoch 7/10
450/450 [==============================] - 5s 11ms/step - loss: 0.3287 - val_loss: 0.3655
Epoch 8/10
450/450 [==============================] - 5s 11ms/step - loss: 0.3151 - val_loss: 0.3732
Epoch 9/10
450/450 [==============================] - 5s 11ms/step - loss: 0.3076 - val_loss: 0.3780
Epoch 10/10
450/450 [=========================

## 保存模型

In [59]:
import os
seq2seq_model_name = "seq2seq.h5"

model.save(os.path.join(model_path,seq2seq_model_name))

## 预测

In [13]:
import os

model = keras.models.load_model(os.path.join(model_path,seq2seq_model_name))

### 编码器

In [17]:
encoder_input_layer = model.input[0]
print(encoder_input_layer)

encoder_state_list = list()
for i in range(len(encoder_units)):
    layer = model.get_layer("Encoder_LSTM_{}".format(i + 1))
    encoder_state_list.append((layer.output[1],layer.output[2]))

# print(encoder_state_list)

encoder_output_states = list()
for v in encoder_state_list:
    encoder_output_states += v
encoder = keras.Model(inputs = encoder_input_layer, outputs= encoder_output_states)

Tensor("Encoder_Input_1:0", shape=(?, 100), dtype=float32)


In [121]:
layer.output_shape

[(None, 100, 128), (None, 128), (None, 128)]

In [122]:
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Encoder_Input (InputLayer)   (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 300)          399000    
_________________________________________________________________
Encoder_LSTM_1 (CuDNNLSTM)   [(None, 100, 128), (None, 220160    
_________________________________________________________________
Encoder_LSTM_2 (CuDNNLSTM)   [(None, 100, 128), (None, 132096    
_________________________________________________________________
Encoder_LSTM_3 (CuDNNLSTM)   [(None, 100, 128), (None, 132096    
Total params: 883,352
Trainable params: 883,352
Non-trainable params: 0
_________________________________________________________________


### 解码器

In [123]:
decoder_input.shape
# decoder_input.set_shape([1,None,1330])

decoder_input.set_shape?

In [14]:
decoder_input = model.input[1]

decoder_state_input_list = []
decoder_state_list = []
for i,unit in enumerate(decoder_units):
    decoder_state_input_h = keras.layers.Input(shape=(unit,),name="State_h_Input_{}".format(i + 1))
    decoder_state_input_c = keras.layers.Input(shape=(unit,),name="State_c_Input_{}".format(i + 1))
    decoder_states_input = [decoder_state_input_h, decoder_state_input_c]
    decoder_state_input_list.append(decoder_states_input)

last_lstm_layer = decoder_input
for i in range(len(decoder_units)):
    lstm_layer = model.get_layer("Decoder_LSTM_{}".format(i + 1))
    
    decoder_output,state_h,state_c = lstm_layer(
            inputs = last_lstm_layer,
            initial_state=decoder_state_input_list[i]
        )
    decoder_state_list.append((state_h,state_c))
    last_lstm_layer = decoder_output


hidden1 = model.get_layer("Decoder_hidden_{}".format(1))
last_layer = last_lstm_layer
for i in range(len(hidden_units)):
    hidden_layer = model.get_layer("Decoder_hidden_{}".format(i + 1))
    output = hidden_layer(inputs = last_layer)
    last_layer = output

c = list()
for a in decoder_state_input_list:
    c += a
decoder_state_input_list = c
states = []
for i in decoder_state_list:
    states += i
decoder_state_list = states
decoder = keras.Model([decoder_input]+c,[output] + decoder_state_list)

    

In [15]:
decoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder_Input (InputLayer)      (10, None, 1330)     0                                            
__________________________________________________________________________________________________
State_h_Input_1 (InputLayer)    (None, 128)          0                                            
__________________________________________________________________________________________________
State_c_Input_1 (InputLayer)    (None, 128)          0                                            
__________________________________________________________________________________________________
Decoder_LSTM_1 (CuDNNLSTM)      [(10, None, 128), (N 747520      Decoder_Input[0][0]              
                                                                 State_h_Input_1[0][0]            
          

In [129]:
decoder.

In [18]:
seq = [113,260,195]
seq = [113,260,195] + [0] * 100
seq = seq[:100]

s = [seq]
s = np.asarray(s)
s.shape
p = encoder.predict(s)



In [31]:
# len(p[0])
c = []
for i in range(6):
    q = np.zeros([1,128])
    c.append(q)

In [32]:
# print(p[0].shape)
se = np.zeros((1,1,voc_size))
se[0,0,89] = 1
a = [se] + c 
# print(a)
print(se.shape)
# decoder.predict(a,batch_size=1,)
decoder.predict(a)

(1, 1, 1330)


InvalidArgumentError: You must feed a value for placeholder tensor 'Encoder_Input_1' with dtype float and shape [?,100]
	 [[{{node Encoder_Input_1}} = Placeholder[dtype=DT_FLOAT, shape=[?,100], _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[{{node Decoder_LSTM_1_2/strided_slice_17/_357}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1353_Decoder_LSTM_1_2/strided_slice_17", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [20]:

# keras.utils.plot_model(decoder, to_file='../images/decoder.png', show_shapes=True, show_layer_names=True)

### 获取回答语句

In [61]:
decoder.input

[<tf.Tensor 'Decoder_Input_1:0' shape=(10, ?, 1330) dtype=float32>,
 <tf.Tensor 'State_h_Input_1_1:0' shape=(?, 128) dtype=float32>,
 <tf.Tensor 'State_c_Input_1_1:0' shape=(?, 128) dtype=float32>,
 <tf.Tensor 'State_h_Input_2_1:0' shape=(?, 128) dtype=float32>,
 <tf.Tensor 'State_c_Input_2_1:0' shape=(?, 128) dtype=float32>,
 <tf.Tensor 'State_h_Input_3_1:0' shape=(?, 128) dtype=float32>,
 <tf.Tensor 'State_c_Input_3_1:0' shape=(?, 128) dtype=float32>]

In [54]:
tokenizer.word_index["怎么"]
# " ".join(jieba.cut(""))
# t

56

In [69]:
def answer(input_seq,max_length):
    """回答
    Args:
        input_seq: str
            问题，输入的话
        max_length: int
            最大回答长度
    Return: str
        回答的话
    """
    words = list(jieba.cut(input_seq))
    words = [tokenizer.word_index.get(word,0) for word in words]
    words = words + [0] * seq_length
    words = words[:seq_length]
    q_nums = np.asarray([words])
    
    encoder_states = encoder.predict(q_nums)
#     print(encoder_states[0])
    decoder_seq = np.zeros((1,1,voc_size))
#     print(decoder_seq.shape)
    decoder.predict([decoder_seq] + encoder_states)

answer("怎么了",100)

InvalidArgumentError: You must feed a value for placeholder tensor 'Encoder_Input_1' with dtype float and shape [?,100]
	 [[{{node Encoder_Input_1}} = Placeholder[dtype=DT_FLOAT, shape=[?,100], _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]
	 [[{{node Decoder_LSTM_3_5/strided_slice_17/_371}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_2594_Decoder_LSTM_3_5/strided_slice_17", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]